### Your name:

<pre> Ashutosh Deowanshi </pre>

### Collaborators:

<pre> Rishab Kamshetty </pre>


In this exercise you will build a decision tree model for classification

In [1]:
import os
import tarfile
from six.moves import urllib
import numpy as np

DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    if not os.path.isdir(housing_path):
        os.makedirs(housing_path)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()

In [2]:
fetch_housing_data()

In [3]:
import pandas as pd

def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)

In [4]:
housing = load_housing_data()

### Fix the categories in the categorical variable

In [5]:
d = {'<1H OCEAN':'LESS_1H_OCEAN', 'INLAND':'INLAND', 'ISLAND':'ISLAND', 'NEAR BAY':'NEAR_BAY', 'NEAR OCEAN':'NEAR_OCEAN'}
housing['ocean_proximity'] = housing['ocean_proximity'].map(lambda s: d[s])

### Add 2 more features

In [6]:
housing["rooms_per_household"] = housing["total_rooms"]/housing["households"]
housing["population_per_household"]=housing["population"]/housing["households"]

### Fix missing data

In [7]:
median = housing["total_bedrooms"].median()
housing["total_bedrooms"].fillna(median, inplace=True) 

### Create dummy variables based on the categorical variable

In [8]:
one_hot = pd.get_dummies(housing['ocean_proximity'])
housing = housing.drop('ocean_proximity', axis=1)
housing = housing.join(one_hot)

In [9]:
housing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 16 columns):
longitude                   20640 non-null float64
latitude                    20640 non-null float64
housing_median_age          20640 non-null float64
total_rooms                 20640 non-null float64
total_bedrooms              20640 non-null float64
population                  20640 non-null float64
households                  20640 non-null float64
median_income               20640 non-null float64
median_house_value          20640 non-null float64
rooms_per_household         20640 non-null float64
population_per_household    20640 non-null float64
INLAND                      20640 non-null uint8
ISLAND                      20640 non-null uint8
LESS_1H_OCEAN               20640 non-null uint8
NEAR_BAY                    20640 non-null uint8
NEAR_OCEAN                  20640 non-null uint8
dtypes: float64(11), uint8(5)
memory usage: 1.8 MB


### Partition into train and test

Use train_test_split from sklearn.model_selection to partition the dataset into 70% for training and 30% for testing.

You can use the 70% for training set as both training and validation by using cross-validation.

In [10]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(housing, test_size=0.3, random_state=42)

### Features

In [11]:
target = 'median_house_value'
features = list(train_set.columns)
features = [f for f in features if f!=target]

In [12]:
X_tr = train_set[features]
y_tr = train_set[[target]]

X_te = test_set[features]
y_te = test_set[[target]]

In [13]:
y_tr_b = 1*np.ravel(y_tr>=179700.0)
y_te_b = 1*np.ravel(y_te>=179700.0)

#### 1) Use grid search with cross-validation (with the help of the GridSearchCV class) to find good hyperparameter values (for parameters 'max_leaf_nodes', 'min_samples_split', 'max_depth') for a DecisionTreeClassifier. You should get around 87% average accuracy across the n-folds.

### Use CV=10 in GridSearchDV

In [14]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier


params = {'max_leaf_nodes': list(range(2, 100)),
          'min_samples_split': [2,3,4,7,10,12], 
          'max_depth':range(2,100) }
grid_search_cv = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1, verbose=1, cv=10)

grid_search_cv.fit(X_tr, y_tr_b)

Fitting 10 folds for each of 3528 candidates, totalling 35280 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  62 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done 662 tasks      | elapsed:    9.7s
[Parallel(n_jobs=-1)]: Done 1662 tasks      | elapsed:   21.2s
[Parallel(n_jobs=-1)]: Done 3062 tasks      | elapsed:   38.4s
[Parallel(n_jobs=-1)]: Done 4862 tasks      | elapsed:   59.6s
[Parallel(n_jobs=-1)]: Done 7062 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 9662 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 12662 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done 16062 tasks      | elapsed:  4.9min
[Parallel(n_jobs=-1)]: Done 19862 tasks      | elapsed:  6.4min
[Parallel(n_jobs=-1)]: Done 24062 tasks      | elapsed:  8.1min
[Parallel(n_jobs=-1)]: Done 28662 tasks      | elapsed: 10.5min
[Parallel(n_jobs=-1)]: Done 33662 tasks      | elapsed: 13.1min
[Parallel(n_jobs=-1)]: Done 35280 out of 35280 | elapsed: 14.1min finished


GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=DecisionTreeClassifier(class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort=False, random_state=42,
                                              splitter='best'),
             iid='warn', n_jobs=-1,
             param_grid={'max_depth': [2, 3, 4, 5, 6, 10],
                         'max_leaf_nodes': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,


In [15]:
from sklearn.metrics import accuracy_score

y_pred = grid_search_cv.predict(X_tr)
accuracy_score(y_tr_b, y_pred)

0.8942414174972314

#### 2) Measure the performance of your best model on the test set

In [16]:
from sklearn.metrics import accuracy_score

y_pred = grid_search_cv.predict(X_te)
accuracy_score(y_te_b, y_pred)

0.872093023255814

### Submit your notebook

Submit your solution on Canvas